In [1]:
import numpy as np
import torch
import psutil
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys
import pickle
from torch.optim import SGD

In [2]:
sys.path.insert(1, '/home/ubuntu/Intents-Analysis/Analysis')
#sys.path.insert(1, '/Users/manjugupta/Desktop/CMU_Courses/Intents/getting_intents/Analysis')

In [3]:
from get_vocab import load_data, get_vocab
from get_frequency import get_frequency

In [4]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is True
8


In [5]:
##Needed Functions
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output


def create_vocabulary(train_file):
    '''This function creates an indexed vocabulary dictionary from the training file'''
    
    vocab, _ = get_vocab(1, train_file)
    
    phone_to_idx = {'unk': 1}#Padding indx = 0, unkown_idx = 1, indexing starts from 2
    for i, phone in enumerate(vocab):
        phone_to_idx[phone] = i + 2
        
    return phone_to_idx

In [6]:
class MyDataset(Dataset):
    def __init__(self, data_file, intent_labels, phone_to_idx):
        data = load_data(data_file)
        self.all_data = []
        
        for intent in data:
            for utterance in data[intent]:
                utterance_to_idx = []
                
                for phone in utterance:
                    if phone not in phone_to_idx:
                        phone = 'unk'
    
                    utterance_to_idx.append(phone_to_idx[phone])
                
                self.all_data.append([utterance_to_idx, intent_labels[intent]])
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):
        input_vector = self.all_data[index][0]
        label = self.all_data[index][1]

        return input_vector, label

In [7]:
def collate_indic(tuple_lst):

    x_lst = [x[0] for x in tuple_lst]
    y_lst = [x[1] for x in tuple_lst]

    # collate x
    B = len(tuple_lst)#Number of training samples
    T = max(len(x) for x in x_lst)#Max length of a sentence

    # x values
    x = torch.zeros([B, T], dtype=torch.int64)
    lengths = torch.zeros(B, dtype=torch.int64)

    for i, x_np in enumerate(x_lst):
        lengths[i] = len(x_np)
        x[i,:len(x_np)] = torch.tensor(x_np)

    # collate y
    y = torch.zeros([B, 6])
    for i, y_label in enumerate(y_lst):
        y[i][y_label] = 1
        
    ids = torch.argsort(lengths, descending=True)

    return x[ids], lengths[ids], y[ids]

In [15]:
#Defining constants and labels
intent_labels = {'movie-tickets':0, 'auto-repair':1, 'restaurant-table':2, 'pizza-ordering':3, 'uber-lyft':4, 'coffee-ordering':5}
train_language = 'gujarati_marathi_bengali_hindi_15'
test_language = 'hindi'

#Loading data
train_file = '/home/ubuntu/Intents-Analysis/TaskMasterData/Get_Phones_Combos/3_lang_variations/taskmaster_training_' + train_language + '.pkl'
test_file = '/home/ubuntu/Intents-Analysis/TaskMasterData/Get_Phones_Combos/1_language/taskmaster_testing_' + test_language + '.pkl'

#create vocabulary and phone_to_idx
phone_to_idx = create_vocabulary(train_file)
print(len(phone_to_idx))

68


In [16]:
train_dataset = MyDataset(train_file, intent_labels, phone_to_idx)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args, collate_fn=collate_indic)

test_dataset = MyDataset(test_file, intent_labels, phone_to_idx)
test_loader_args = dict(shuffle=False, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
valid_loader = DataLoader(test_dataset, **test_loader_args, collate_fn=collate_indic)

In [17]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size=69, embed_size=128, hidden_size=128, label_size=6):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(embed_size*3)

        self.lstm = nn.LSTM(embed_size*3, hidden_size, num_layers=2)
        self.linear = nn.Linear(hidden_size, label_size)

    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """

        # B,T,H
        input = self.embed(x)

        # (B,T,H) -> (B,H,T)
        input = input.transpose(1,2)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        # (B,H,T)
        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)

        output, (hn, cn) = self.lstm(pack_tensor)

        #output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)

        #output = torch.cat([hn[0], hn[1]], dim=1)
        logits = self.linear(hn[0])

        return logits

In [22]:
model = RNNClassifier()
opt = optim.Adam(model.parameters(), lr = 0.001)
#opt = SGD(model.parameters(), lr=0.05)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)

RNNClassifier(
  (embed): Embedding(69, 128)
  (cnn): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (cnn2): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (cnn3): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
  (batchnorm): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(384, 128, num_layers=2)
  (linear): Linear(in_features=128, out_features=6, bias=True)
)

In [23]:
print(train_language, test_language)
max_acc = 0

for i in range(1000):
    #print("epoch ", i)
    loss_accum = 0.0
    batch_cnt = 0

    acc_cnt = 0
    err_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader):

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        opt.zero_grad()
        
        logits = model(x, lengths)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()

        out_val, out_indices = torch.max(logits, dim=1)
        target_val, tar_indices = torch.max(y, dim=1)

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    print("train acc: ", acc_cnt/(err_cnt+acc_cnt), " train loss: ", loss_accum / batch_cnt, '--time:', time.time() - start_time)

    model.eval()
    acc_cnt = 0
    err_cnt = 0

    #start_time = time.time()
    for x, lengths, y in valid_loader:

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        
        logits = model(x, lengths)

        out_val, out_indices = torch.max(logits, dim=1)
        target_val, tar_indices = torch.max(y, dim=1)

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    current_acc = acc_cnt/(err_cnt+acc_cnt)
    if current_acc > max_acc:
        max_acc = current_acc
                
    print("validation: ", current_acc, '--max', max_acc, '--time:', time.time() - start_time)

gujarati_marathi_bengali_hindi_15 hindi
train acc:  0.24872579001019368  train loss:  0.4611376770164656 --time: 3.5377166271209717
validation:  0.18666666666666668 --max 0.18666666666666668 --time: 4.207354784011841
train acc:  0.30139313625552155  train loss:  0.4328637291555819 --time: 3.1559157371520996
validation:  0.15333333333333332 --max 0.18666666666666668 --time: 3.8269591331481934
train acc:  0.3143051308188923  train loss:  0.42676447526268335 --time: 4.210057258605957
validation:  0.19333333333333333 --max 0.19333333333333333 --time: 4.713374614715576
train acc:  0.3384301732925586  train loss:  0.4203515597011732 --time: 4.15291166305542
validation:  0.19333333333333333 --max 0.19333333333333333 --time: 4.537609577178955
train acc:  0.3696907917091403  train loss:  0.4110347576763319 --time: 4.836566686630249
validation:  0.21666666666666667 --max 0.21666666666666667 --time: 5.256062269210815
train acc:  0.4148827726809378  train loss:  0.3944229807542718 --time: 4.865340

validation:  0.7366666666666667 --max 0.7366666666666667 --time: 5.072945833206177
train acc:  1.0  train loss:  0.002691733625555492 --time: 4.436884164810181
validation:  0.7433333333333333 --max 0.7433333333333333 --time: 5.0824432373046875
train acc:  1.0  train loss:  0.001960788144851508 --time: 4.562573671340942
validation:  0.7366666666666667 --max 0.7433333333333333 --time: 5.216519832611084
train acc:  1.0  train loss:  0.0017082726825838504 --time: 4.494392156600952
validation:  0.7333333333333333 --max 0.7433333333333333 --time: 5.110377073287964
train acc:  1.0  train loss:  0.0016286379515962756 --time: 4.498527765274048
validation:  0.7466666666666667 --max 0.7466666666666667 --time: 5.125490665435791
train acc:  1.0  train loss:  0.0014541093513126607 --time: 4.424524307250977
validation:  0.7466666666666667 --max 0.7466666666666667 --time: 4.951566219329834
train acc:  1.0  train loss:  0.001379696318231847 --time: 4.44624137878418
validation:  0.7466666666666667 --max

KeyboardInterrupt: 